# Exemplo 1 - Redução de dimensão

Neste exemplo iremos analisar dados agregados do último censo.

Será que conseguiremos resumir os dados apresentados em poucas visualizações?

## Carregar pacotes

In [ ]:
library(tidyverse)
library(magrittr)
library(plotly)
library(psych)

## Carregar dados

In [ ]:
dados_municipios <- read_csv("/home/vm-data-science/dados/dados_municipios.csv")

In [ ]:
dados_municipios %>% head

## Análises

### Análise exploratória

- Visão geral

In [ ]:
dados_municipios %>% 
  summarise_if( is.double, mean )

- Visão por estado

In [ ]:
dados_municipios %>%
  group_by( estado ) %>% 
  summarise_if( is.double, mean )

### Algumas visualizações

In [ ]:
dados_municipios %>% 
  plot_ly( x = ~estado,
          y = ~t_agua,
          type = 'box' ) 

In [ ]:
dados_municipios %>% 
  plot_ly( x = ~estado,
           y = ~t_lixo,
           type = 'box' )

In [ ]:
dados_municipios %>% 
  plot_ly( x = ~estado,
           y = ~t_luz,
           type = 'box' ) 

In [ ]:
dados_municipios %>% 
  plot_ly( x = ~estado,
           y = ~T_ANALF18M,
           type = 'box' ) 

### Analisar correlações

In [ ]:
correlacao <- dados_municipios %>% 
  select( contains('t_') ) %>% 
  cor() %>% 
  round(., 2)

correlacao

In [ ]:
plot_ly( z = correlacao,
         x = colnames(correlacao),
         y = colnames(correlacao), 
         colorscale = "Greys", 
         type = "heatmap" )

Aparentemente observamos 2 grupos de índices: índices de educação e índices de qualidade de moradia.

### *Principal Components Analysis* - PCA

Agora que podemos criar 2 índices, podemos reduzir a dimensão do dataset em somente 2 colunas.

In [ ]:
pca_model <- principal(dados_municipios %>% 
                        select( contains('t_') ), 
                       nfactors = 2, 
                       rotate = "varimax" )

In [ ]:
pca_model

- São muitas informações. Iremos focar nos valores que chamamos de "cargas".

In [ ]:
pca_model$loadings[]

Esta análise nos apresenta a possibilidade de criarmos os 2 índices esperados.

A coluna RC1 representam cargas altas e positivas para os índices relacionados a educação.

A coluna RC2 representam cargas altas e positivas para os índices relacionados a moradia.

### Obtenção dos índices e análises

Os índices também são calculados pela ferramenta. 

Um ponto a destacar é que os índices estão padronizados para terem média igual a zero e desvio-padrão igual a 1.

Vamos adicionar estes índices em nosso banco de dados.

In [ ]:
dados_municipios %<>% 
  mutate( indice_atraso_escolar = pca_model$scores[,1],
          indice_qualidade_moradia = pca_model$scores[,2] )

dados_municipios %>% 
    head

In [ ]:
dados_municipios %>% 
    select( contains('indice') ) %>% 
    gather( key = indices, value = valores ) %>% 
    group_by( indices ) %>% 
    summarise( media = round( mean(valores) ),
               desvio_padrao = sd(valores) )

- Agora que temos 2 índices que resumem todo o dataset, podemos fazer uma análise simples e resumida.

In [ ]:
dados_municipios %>% 
  group_by( estado ) %>% 
  summarise( indice_qualidade_moradia = mean(indice_qualidade_moradia), 
             indice_atraso_escolar = mean(indice_atraso_escolar) ) %>% 
  plot_ly( x = ~indice_atraso_escolar,
           y = ~indice_qualidade_moradia,
           marker = list( size = 10),
           type = 'scatter',
           mode = 'markers',
           text = ~paste( "Estado: ", estado ),
           hoverinfo = 'text',
           showlegend = FALSE )

- Podemos melhorar um pouco mais esta visualização.

In [ ]:
dados_municipios %>% 
  group_by( estado ) %>% 
  summarise( indice_qualidade_moradia = mean(indice_qualidade_moradia), 
             indice_atraso_escolar = mean(indice_atraso_escolar) ) %>% 
  mutate( d_qual = ifelse( indice_qualidade_moradia > 0, 'moradia_alta', 'moradia_baixa'),
          d_atra = ifelse( indice_atraso_escolar > 0, 'atraso_alto', 'atraso_baixo'),
          quadrante = paste0(d_qual, ' - ', d_atra) ) %>% 
  plot_ly( x = ~indice_atraso_escolar,
           y = ~indice_qualidade_moradia,
           color = ~quadrante,
           marker = list( size = 15),
           type = 'scatter',
           mode = 'markers',
           text = ~paste( "</br> Estado: ", estado,
                          "</br> Moradia: ", d_qual,
                          "</br> Atraso: ", d_atra),
           hoverinfo = 'text',
           showlegend = TRUE ) %>% 
  layout( xaxis = list( title = "Índice de atraso escolar" ), 
          yaxis = list( title = "Índice de qualidade de moradia" ) )